### Importing Libraries and Reviewing Lab Notes

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
#import spacy
from tqdm import tqdm
import urllib
from urllib.request import urlopen
from nltk.stem import WordNetLemmatizer 
# lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [140]:
sw = stopwords.words('english')
remove_punctuation = string.punctuation + '..' + '...' + "''" + "``"
from collections import Counter
import math
from collections import defaultdict

In [63]:
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome(executable_path=ChromeDriverManager.install())

In [ ]:
with open(filename, encoding='utf8') as infile:
    html = BeautifulSoup(infile, "html.parser")

In [71]:
page = requests.get("http://www.goodreads.com/book/show/9222475-infernal-devices")
page.content
with open(page.content, encoding='utf8') as infile:
    soup = BeautifulSoup(infile, "html.parser")
soup.prettify
f = open("page.html", "w")
f.write(soup.prettify())
f.close()

In [49]:
nlp = spacy.load('en')

In [50]:
def scrap_book(href, driver, nlp):
    driver.get(href)       
    time.sleep(5)
    page_soup = BeautifulSoup(driver.page_source, features="lxml")
    book_title = page_soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
    author = page_soup.find_all('span', itemprop='name')[0].contents[0]
    description = ' '.join([c for c in page_soup.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)])
    ratings = page_soup.find_all('a', href="#other_reviews")
    rating_count = -1
    rating = -1
    for raiting in ratings:
        if raiting.find_all('meta', itemprop="ratingCount"):
            rating_count = raiting.text.replace('\n', '').strip().split(' ')[0]
        elif raiting.find_all('meta', itemprop="reviewCount"):
            rating = raiting.text.replace('\n', '').strip().split(' ')[0]
    doc = nlp(description)
    token_list = [token for token in doc if not token.is_stop and not token.is_punct]
    return (book_title, author, rating_count, rating, description, token_list)

## 1. Data collection
### 1.1. Get the list of books

#### Firstly we will be working with one page to understand how it has to  be done. Since goodreads is blocked for some reason in Kazakhstan, I will try to investigate the goodreads by using 'requests' library. I will save then it to html to see how it looks.

In [81]:
page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1")
soup_page = BeautifulSoup(page.content, features="lxml")
url = soup_page.find_all('div', class_="js-tooltipTrigger tooltipTrigger") # here we are setting some parameters to choose books
# as a result we obtain a dictionary for all first 100 books

In [77]:
f = open("page.html", "w")
f.write(soup_page.prettify())
f.close()

For example, here we can see the first book, 'Hunger Games'. We have to work with href part. 

In [76]:
url[0] 

<div class="js-tooltipTrigger tooltipTrigger" data-resource-id="2767052" data-resource-type="Book">
<a href="/book/show/2767052-the-hunger-games" title="The Hunger Games">
<img alt="The Hunger Games (The Hunge..." class="bookCover" itemprop="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052._SY75_.jpg"/>
</a> </div>

#### Thus, it becomes clear that we have to run for loops two times - for all 100 urls of first 300 pages.Then, we will have 30, 000 books.

Before running our for loop, let's check our logic for one url:

In [101]:
url_list = []
url_str = str(url[0]) 
url_str = url_str.split(" ")
url_str = url_str[5] 
url_str = url_str.split("\"")[1]
url_list += ["https://www.goodreads.com" + url_str]
url_list   #looks alright 

['https://www.goodreads.com/book/show/2767052-the-hunger-games']

In [100]:
url_list = []
top_books = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="
for _ in range (1,301):   # pages
    page = requests.get(top_books + str(_))
    soup_page = BeautifulSoup(page.content, features="lxml")
    url = soup.find_all('div', class_="js-tooltipTrigger tooltipTrigger")
    
    for i in range (0,100): # books
        url_str = str(url[i]) 
        url_str = url_str.split(" ")
        url_str = url_str[5] 
        url_str = url_str.split("\"")[1]
        url_list += ["https://www.goodreads.com" + url_str]

In [111]:
with open('url_list.txt', 'w') as f:
    for item in url_list:
        f.write("%s\n" % item) # saving it to txt

Now, we have saved our list to a  txt file with 30 thousand books. 

### 1.2. Book crawling

#### In this section I am saving all urls to htmls. This is a part that took the most memory and time. Therefore, I used multiprocessing

In [12]:
df= pd.read_csv('url_list.txt', names=['url'], header=None)
df.head()

,url
0,https://www.goodreads.com/book/show/2767052-th...
1,https://www.goodreads.com/book/show/2.Harry_Po...
2,https://www.goodreads.com/book/show/2657.To_Ki...
3,https://www.goodreads.com/book/show/1885.Pride...
4,https://www.goodreads.com/book/show/41865.Twil...


In [16]:
# for idx, link in list(enumerate(df['url'][4:20])):
#     urllib.request.urlretrieve(link, r'C:\\Users\\User\\Desktop\\DS.Sapienza\\Data Mining. Aris\\HW3\\mein_arbeit\\html_data\\{}_book_{}.html'.format(link.split('/')[-1],str(idx)))    

In [171]:
def cpu_usage(idx, link):
    urllib.request.urlretrieve(link, r"/home/hduser/nfs/IliyasSpace/HW_Sapienza/Algorithmic Methods Of Data Mining And Laboratory (Aris)/HW3/html_data/{}_book_{}.html".format(link.split('/')[-1],str(idx)))

In [163]:
cpu_usage(4, 'https://www.goodreads.com/book/show/41865.Twilight')

('/home/hduser/nfs/IliyasSpace/HW_Sapienza/Algorithmic Methods Of Data Mining And Laboratory (Aris)/HW3/html_data/41865.Twilight_book_4.html',
 <http.client.HTTPMessage at 0x7f8e05fba278>)

In [172]:
with Pool(cpu_count() - 6) as pool:
    for idx, link in list(enumerate(df['url'][0:30000])):
        pool.apply_async(cpu_usage, (idx, link))
    pool.close()
    pool.join()
    print('Done')

Done


After three hours we get almost all of our html files

### 1.3 Parse downloaded pages

#### In this part we are iterating through our html files to create a dataframe that will further be used for analysis

In [33]:
def information_taking(file):      
    f = open(file, encoding='utf8')
    
#     with open(file, encoding='utf8') as infile:
    soup = BeautifulSoup(f.read(), features='lxml')  
    
    f.close()
    
    
    Series = soup.find(id="bookSeries")
    if Series:
        Series = Series.text.replace('(', '').replace(')', '').strip()
    
    page_number = soup.find(itemprop="numberOfPages")
    if page_number:
        page_number = int(page_number.text.split()[0])
  
    Authors = soup.find('span', itemprop='name')
    if Authors:
        Authors = Authors.text
    else:
        print('', file)
        return
    
    bookTitle = soup.find(id="bookTitle")
    if bookTitle:
        bookTitle = bookTitle.text.replace('\n', '').strip()
    else:
        print('', file)
        return


    rating = soup.find(itemprop="ratingValue")
    if rating:
        rating = float(soup.find(itemprop="ratingValue").text)
    ratingCount = soup.find(itemprop="ratingCount")
    if ratingCount:
        ratingCount = int(ratingCount.get('content'))
  
    reviewCount = soup.find(itemprop="reviewCount")
  
    if reviewCount: 
        reviewCount = int(reviewCount.get('content'))
    Plot = soup.find(id="description")
    if Plot:
        Plot = (Plot).find_all('span')[-1].text 
        try:
            if detect(Plot) != 'en': #check if is a English plot
                return
        except:
              pass

    try:
        publish = (soup.find(id="details")).find_all('div')[1].text.split('\n')[2].strip()
    except:
        publish = None
    try:
        characters = ', '.join(map (lambda x : x.text,  (soup.find(id="bookDataBox")).find_all('a', href = re.compile("^.characters"))))
    except:
        characters = None
    try:    
        settings = ', '.join(map (lambda x : x.text.replace(',',''),  (soup.find(id="bookDataBox")).find_all('a', href = re.compile("^.places"))))
    except:
        settings = None
    try:
        link = soup.find('link', itemprop='url').get('href')
    except:
        link = None
        
        
    return [bookTitle, Series, Authors, rating, Plot, ratingCount, reviewCount, page_number, publish, characters, settings, link]

In [348]:
information_taking("/home/hduser/nfs/IliyasSpace/HW_Sapienza/Algorithmic Methods Of Data Mining And Laboratory (Aris)/HW3/html_data2/13069215-marshmallow-madness_book_29988.html")

['Marshmallow Madness!: Dozens of Puffalicious Recipes',
 '',
 'Shauna Sever',
 4.21,
 'If you’ve never tasted a fresh, homemade marshmallow, are you in for a treat! Marshmallow Madness! shows how to whip up dozens of fluffy, puffy flavors—from Strawberry and Vanilla to Buttered Rum, Root Beer Float, Maple Bacon, and more. Author Shauna Sever also includes easy recipes for homemade graham crackers, drinks for mallow dunking, and a host of irresistible desserts, including Lemon Dream Whoopie Pies, Blonde Rocky Road, and S’mores Cupcakes.',
 273,
 20,
 96,
 'February 28th 2012',
 '',
 '',
 'https://www.goodreads.com/review/show/303120196']

In [41]:
# content_list = []
# directory = 'C:\\Users\\User\\Desktop\\DS.Sapienza\\Data Mining. Aris\\HW3\\mein_arbeit\\html_data'
# for content in os.listdir(directory):
#     content_list.append(os.path.join(directory,content))

#### We have to create a list of unique pathes of our html files in order to iterate them through parsing function

In [160]:
content_list = []
directory = "/home/hduser/nfs/IliyasSpace/HW_Sapienza/Algorithmic Methods Of Data Mining And Laboratory (Aris)/HW3/html_data"
for content in os.listdir(directory):
    content_list.append(os.path.join(directory,content))

In [60]:
df_final = pd.DataFrame(columns  = ['bookTitle','bookSeries', 'bookAuthors', \
                                    'ratingValue', 'Plot', 'ratingCount','reviewCount' ,\
                                    'NumberofPages','PublishingDate', 'Characters','Setting', 'url'])
#[bookTitle, bookSeries, bookAuthors, ratingValue, Plot, ratingCount, reviewCount, Number_of_Pages,Published,Characters,Settings,link]

In [351]:
with Pool(cpu_count() - 25) as pool:
    df_final = pd.DataFrame(columns  = ['bookTitle','bookSeries', 'bookAuthors', \
                                    'ratingValue', 'Plot', 'ratingCount','reviewCount' ,\
                                    'NumberofPages','PublishingDate', 'Characters','Setting', 'url'])
    for book in content_list:
        pool.apply_async(extract_info, (df_final, book))
        
    pool.close()
    pool.join()
    print('Done')

In [61]:
for book in content_list:
    info = information_taking(book)
    a_series = pd.Series(info, index = df_final.columns)
    df_final = df_final.append(a_series, ignore_index=True)

#### Finally, we will gather information about all our htmls into one dataframe again either by using multiprocessing or just using for loops.

## 2. Search Engine

#### Since it took too much space, I saved all functions for this section in a separate python file which you will see in a directory

This part is basically cleaning the information about plot in data. It removes stops and punctuations. It does the stemming and etc. 

In [73]:
tqdm.pandas()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [131]:
df_final['tokenized_dscr'] = df_final['Plot'].progress_apply(lambda x: prepare_search(x))





100%|██████████| 15/15 [00:00<00:00, 234.47it/s]


### 2.1.1) Create your index!

To create our inverse index we have to firstly store in the words list all the token we have met tokenizing the plot. Then, sorting by alphabetical order is done.
In the dictionary, we create a key-value relationship so that we have a token as a key and an index of that particular word as a value.

In [149]:
inv_idx, voc, vocab = dict_prep(df_final['tokenized_dscr'])





100%|██████████| 15/15 [00:00<00:00, 2148.50it/s]




100%|██████████| 804/804 [00:00<00:00, 803866.61it/s]




100%|██████████| 804/804 [00:00<00:00, 804058.28it/s]




15it [00:00, ?it/s]




15it [00:00, 15004.66it/s]


### 2.1.2) Execute the query

Based on our token and key information, we are able to find common words in plots. Basically,this is what this function does

In [108]:
query('love', df_final, inv_idx, voc).head()





100%|██████████| 1/1 [00:00<?, ?it/s]


,bookTitle,Plot,url
1,The Fault in Our Stars,Despite the tumor-shrinking medical miracle th...,https://www.goodreads.com/review/show/289575457
4,The Book Thief,Librarian's note: An alternate cover edition c...,https://www.goodreads.com/review/show/387598058
6,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,https://www.goodreads.com/review/show/94754648
7,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/review/show/122569940
9,The Giving Tree,"""Once there was a tree...and she loved a littl...",https://www.goodreads.com/review/show/133528388


### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

We will find all the documents that contains all the words in the query.
We will Sort them by their similarity with the query
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k.

In [112]:
df_final['tokenized_dscr'] = df_final['Plot'].progress_apply(lambda x: prepare_search2(x))
df_final['tokenized_dscr2'] = df_final['tokenized_dscr'].progress_apply(lambda x: Counter(x))





100%|██████████| 15/15 [00:00<00:00, 283.08it/s]




100%|██████████| 15/15 [00:00<00:00, 14775.61it/s]


In [116]:
df_final.tokenized_dscr2.head(5)

0    {'journey': 1, 'end': 1, 'world': 2, 'fantast'...
1    {'despit': 1, 'tumorshrink': 1, 'medic': 1, 'm...
2    {'librarian': 1, 'note': 1, 'altern': 1, 'cove...
3    {'scarlett': 1, 'ohara': 1, 'beauti': 1, 'spoi...
4    {'librarian': 1, 'note': 1, 'altern': 1, 'cove...
Name: tokenized_dscr2, dtype: object

In [127]:
empty_list = []
for n in tqdm(range(len(df_final["tokenized_dscr2"]))):
    empty_list.append(tf_score(df_final["tokenized_dscr2"][n]))
df_final['tf_score'] =empty_list





100%|██████████| 15/15 [00:00<00:00, 15054.93it/s]


In [128]:
df_final.head(2)

,bookTitle,bookSeries,bookAuthors,ratingValue,Plot,ratingCount,reviewCount,NumberofPages,PublishingDate,Characters,Setting,url,tokenized_dscr,tokenized_dscr2,tf_score
0,The Chronicles of Narnia,The Chronicles of Narnia Publication Order #1–7,C.S. Lewis,4.26,"Journeys to the end of the world, fantastic cr...",524435,10441,767,September 16th 2002,"Polly, Aslan, Lucy Pevensie, Edmund Pevensie, ...",London England,https://www.goodreads.com/review/show/2630144,"[journey, end, world, fantast, creatur, epic, ...","{'journey': 1, 'end': 1, 'world': 2, 'fantast'...","{'journey': 0.01020408163265306, 'end': 0.0102..."
1,The Fault in Our Stars,,John Green,4.20,Despite the tumor-shrinking medical miracle th...,3588818,156048,313,January 10th 2012,"Hazel Grace Lancaster, Augustus Waters, Isaac","Indianapolis Indiana, Amsterdam",https://www.goodreads.com/review/show/289575457,"[despit, tumorshrink, medic, miracl, bought, y...","{'despit': 1, 'tumorshrink': 1, 'medic': 1, 'm...","{'despit': 0.019230769230769232, 'tumorshrink'..."


In [137]:
dic = {}
for a in tqdm(df_final['tokenized_dscr']):   
    for b in a:
         dic.update({ b : math.log(len(df_final)/ len(vocab[b]))})   





100%|██████████| 15/15 [00:00<00:00, 7509.50it/s]


### Create the new inverse index

In [150]:
n_inv_idx = new_inverse_index(inv_idx, voc, empty_list, dic)





100%|██████████| 804/804 [00:00<00:00, 402316.92it/s]


### Cosine-similarity score

In [161]:
# ab = np.zeros(len(vocab))
# for w in query:
#     arr_q[voc[w]] = 1
# cos_sim = {}
# for doc in tqdm(s):

#### Above should have been a code for cosine similarity 

## 5. Algorithmic Question

Here we are creating an algorithm so that we have to find the maximum length of a subsequence of characters that is in alphabetical order.

In [154]:
def alg_question(string):
    
    if len(string)==1:
        return [string]
    else:
        teta=alg_question(string[:len(string)-1])
        for a in teta:
            if string[-1]>a[-1]:
                teta.append(a+string[-1])
        teta.append(string[-1])
        return teta

In [159]:
alg_question('qpkldpoqwdpvksdmfsngse')[:10] # information of testing

['q', 'p', 'k', 'kl', 'l', 'd', 'kp', 'klp', 'lp', 'dp']